# Allocating Qubits on QPU Devices

In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

This notebook demonstrates how you can specify explicitly which qubits to use when you run a quantum circuit on QPU devices from Rigetti.

When you submit a circuit for execution on a QPU, Amazon Braket performs a series of compilation steps: it maps the _abstract qubits_ in your circuit to _physical qubits_ in the device; it synthesizes gates into the native gate set of the device; it optimizes the circuit to reduce the number of gates; and finally, it translates the gates into executable pulses.

This section shows how the first step, called qubit allocation, works for the Rigetti Aspen-M-3 device.

In [2]:
# general imports
from braket.aws import AwsDevice
from braket.circuits import Circuit
import numpy as np
import random

## Automatic qubit allocation

Qubit allocation for Rigetti devices on Amazon Braket utilizes [the Quil Compilers](https://pyquil-docs.rigetti.com/en/latest/compiler.html#the-quil-compiler)'s _rewiring_ strategies. By default, when you submit a circuit on Amazon Braket to a Rigetti device, the circuit is rewired according to the [PARTIAL](https://pyquil-docs.rigetti.com/en/latest/compiler.html#partial) rewiring strategy. Specifically, the compiler starts with an empty mapping from logical to physical qubits. Taking into account the latest calibration data of the device, the compiler fills in the mapping with the goal, sequentially, to maximize the overall fidelity of the circuit.

The example that follows shows how to create a GHZ state on qubits that are not physically connected. After the task is completed, you can obtain a list of the actual gates executed on the device, by viewing the result metadata.

First, instantiate the Rigetti Aspen-M-3 device and retrieve its connectivity graph, which shows the qubits that are directly connected on the chip.

In [6]:
device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3")

connectivity_graph = device.properties.paradigm.connectivity.connectivityGraph
print(f"the connectivity of {device.name} is: {connectivity_graph}")

the connectivity of Aspen-M-3 is: {'0': ['1', '7'], '1': ['0', '16', '2'], '10': ['11', '113', '17'], '100': ['101', '107'], '101': ['100', '102', '116'], '102': ['101', '103', '115'], '103': ['102', '104'], '104': ['103', '105', '7'], '105': ['104', '106'], '106': ['105', '107'], '107': ['100', '106'], '11': ['10', '12', '26'], '110': ['111', '117'], '111': ['110', '112', '126'], '112': ['111', '113'], '113': ['10', '112', '114'], '114': ['113', '115', '17'], '115': ['102', '114', '116'], '116': ['101', '115', '117'], '117': ['110', '116'], '12': ['11', '13', '25'], '120': ['121', '127'], '121': ['120', '122', '136'], '122': ['121', '123', '135'], '123': ['122', '124', '20'], '124': ['123', '125', '27'], '125': ['124', '126'], '126': ['111', '125', '127'], '127': ['120', '126'], '13': ['12', '14'], '130': ['131', '137'], '131': ['130', '132', '146'], '132': ['131', '133', '145'], '133': ['132', '134', '30'], '134': ['133', '135', '37'], '135': ['122', '134', '136'], '136': ['121', '13

Next, create a GHZ circuit with three qubits 0, 2, 4, and run it on the Aspen-M-3 device. Notice that none of these qubits are connected on the Aspen-M-3 connectivity graph.

In [7]:
# create a GHZ state with non-neighboring qubits
circuit = Circuit()
circuit.h(0).cnot(0,2).cnot(0,4)
print(circuit)

rigetti_rewiring = device.run(circuit, shots=10)

T  : |0|1|2|
            
q0 : -H-C-C-
        | | 
q2 : ---X-|-
          | 
q4 : -----X-

T  : |0|1|2|


In [8]:
print("Status of task:", rigetti_rewiring.state())

Status of task: CREATED


To verify the final qubit allocation, retrieve the compiled program that was executed:

In [9]:
result = rigetti_rewiring.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'001': 4, '111': 3, '000': 2, '101': 1})
The compiled circuit is:
 DECLARE ro BIT[3]
PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
RZ(-pi/2) 12
RX(-pi/2) 12
RZ(pi) 13
XY(pi) 12 13
RZ(pi/2) 12
RX(pi/2) 12
RZ(-pi/2) 12
XY(pi) 12 13
RZ(pi/2) 25
RX(-pi/2) 25
CZ 25 12
RZ(pi) 12
RX(-pi/2) 13
RX(pi/2) 25
RZ(-pi/2) 25
MEASURE 25 ro[2]
MEASURE 13 ro[1]
MEASURE 12 ro[0]



Notice that the PARTIAL rewiring was applied. The qubits 0, 2, 4 in the original circuit were mapped to three other qubits in the Rigetti device, and the gates were compiled into native gates.

## User-defined qubit allocation

In Amazon Braket, you can choose to prescribe a qubit mapping manually, and prevent further rewiring for Rigetti devices. To enable manual mapping, set `disable_qubit_rewiring=True` when submitting the task to run.

If all the gates in the circuit satisfy the topological constraints of the device, Amazon Braket maps abstract qubit $i$ in the circuit to the physical qubit $i$ in the device, and maps qubit pair $(i, j)$ to the connection $(i, j)$ in the device. On the other hand, Amazon Braket raises an exception if a specified qubit or qubit pair do not exist in the device connectivity graph.

In [11]:
# create a random state with neighboring qubits
q1=random.choice(list(connectivity_graph))
q2=int(connectivity_graph[q1][0])
q1=int(q1)

circuit = Circuit()
circuit.rz(0,np.pi/2).cnot(q1,q2).x(7)
print(circuit)
rigetti_task = device.run(circuit, shots=10, disable_qubit_rewiring=True)

T  : |   0    |1|
                 
q0 : -Rz(1.57)-C-
               | 
q1 : ----------X-
                 
q3 : -X----------

T  : |   0    |1|


In [12]:
print("Status of task:", rigetti_task.state())

Status of task: COMPLETED


In [ ]:
result = rigetti_task.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'011': 6, '001': 2, '101': 1, '000': 1})
The compiled circuit is:
 DECLARE ro BIT[3]
PRAGMA INITIAL_REWIRING "NAIVE"
RESET
RZ(pi) 1
RX(-pi/2) 1
XY(pi) 0 1
RZ(-pi/2) 0
RX(pi/2) 0
RZ(pi/2) 0
XY(pi) 0 1
RX(pi) 3
MEASURE 3 ro[2]
MEASURE 1 ro[1]
MEASURE 0 ro[0]



The qubits in the original circuit followed a one-to-one mapping to the physical qubits in the device. Other compilation steps, such as gate synthesis and circuit optimization, are still performed. These steps allow the circuit to run successfully and improve the overall fidelity.

### Using the qubits with the highest two-qubit gate fidelity

Additionally, the device properties include calibration data, which you can use to find the qubits and qubit pairs with the highest fidelities for particular gates.

The following function finds the qubit pair that has the highest two-qubit fidelity of an input gate, which can be any of the gates native to the Rigetti device, namely CPHASE, XY or CZ.

In [13]:
def find_qubit_pair(gate):
    "Function to find the qubit pair that has the highest gate fidelity of a particular gate"
    
    # check whether the input gate is a string
    if not isinstance(gate, str):
        raise ValueError('The input gate must be a string type.') 
    
    # check whether the input gate is a native gate
    gate_list = ['CPHASE', 'CZ', 'XY']
    if gate not in gate_list:
        raise ValueError('The input gate must be either CPHASE, CZ or XY.')
    
    # load all calibration data from device.properties
    calibration_2Q = device.properties.provider.specs['2Q']
    highest_fidelity = 0

    # iterate through all calibration data to find the highest fidelity
    for pair in calibration_2Q.keys():
        
        # if the particular gate type is supported by the qubit pair
        if ('f'+ gate) in calibration_2Q[pair].keys(): 
            
            if calibration_2Q[pair]['f'+ gate] > highest_fidelity:
                
                # update the highest_fidelity and the best_pair
                highest_fidelity = calibration_2Q[pair]['f'+ gate]
                best_pair = pair

    # generate the two qubits as integers       
    q1 = best_pair[0]
    i = 1
    while best_pair[i] is not '-':
        q1 += best_pair[i]
        i += 1

    q1 = int(q1)
    q2 = int(best_pair[i+1:])
        
    return q1, q2, highest_fidelity

The example in the following code applies a native two-qubit gate on the qubit pair that has the highest fidelity of that gate. 

In [14]:
# the gate must be either 'CZ', 'CPHASE' or 'XY'
gate = 'CZ'
# find the qubit pair with the highest gate fidelity
q1, q2, highest_fidelity = find_qubit_pair(gate)
print('The highest fidelity for '+gate+' gate is:', highest_fidelity)
print(f'And the corresponding qubit pair is: qubit {q1} and qubit {q2}')

# create a circuit with the gate applied to the discovered qubit pair.
# note that CPHASE in Rigetti corresponds to cphaseshift in Braket
circuit = Circuit()
circuit.cz(q1,q2)
print(circuit)
rigetti_task = device.run(circuit, shots=1000, disable_qubit_rewiring=True)

The highest fidelity for CZ gate is: 0.9794230141816626
And the corresponding qubit pair is: qubit 21 and qubit 36
T   : |0|
         
q21 : -C-
       | 
q36 : -Z-

T   : |0|


In [15]:
print("Status of task:", rigetti_task.state())

Measurement counts: Counter({'00': 910, '01': 55, '10': 33, '11': 2})
The compiled circuit is:
 DECLARE ro BIT[2]
PRAGMA INITIAL_REWIRING "NAIVE"
RESET
CZ 21 36
MEASURE 21 ro[1]
MEASURE 36 ro[0]



The qubits in the original circuit followed a one-to-one mapping to the physical qubits in the device. Since only native gates were used, the actual gates executed are the same as the gates in the original circuit.

<div class="alert alert-block alert-info">
<b>Note:</b> The IonQ device does not support manual allocation. For circuits submitted to the IonQ device, qubits are allocated automatically.
</div>

In [16]:
print("Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.2f} USD")

Task Summary
{'arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3': {'shots': 1020, 'tasks': {'QUEUED': 1, 'CREATED': 2}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 1.26 USD
